In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score
import pickle

from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import jsonify, request
import requests
import json

In [2]:
!pip install flask-ngrok

# **Dataset Iris**

In [24]:
dataset = pd.read_csv('/content/drive/My Drive/BCML/datasets/iris.csv')
dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


# **Feature Engineering**

In [25]:
label = dataset['Species'] # Target prediksi
feature = dataset.drop(['Id', 'Species'], axis=1) # Data Id tidak diperlukan

In [26]:
scaler = StandardScaler()
scaler.fit(feature)
feature_scale=scaler.transform(feature)
data_feature = pd.DataFrame(feature_scale, columns=feature.columns)

In [36]:
data_feature

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977
...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956
146,0.553333,-1.281972,0.705893,0.922064
147,0.795669,-0.124958,0.819624,1.053537
148,0.432165,0.800654,0.933356,1.447956


In [27]:
feature_train, feature_test, label_train, label_test = train_test_split(data_feature, label, test_size=0.15)

# **Model**

In [28]:
logreg = LogisticRegression()
logreg.fit(feature_train, label_train)
label_pred = logreg.predict(feature_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(feature_test, label_test))) # .score() -> Accuracy
print(f1_score(label_test, label_pred, average='macro'))
print(precision_score(label_test, label_pred, average='macro'))
print(recall_score(label_test, label_pred, average='macro'))

Accuracy of logistic regression classifier on test set: 1.00
1.0
1.0
1.0


In [29]:
filename = 'model.pkl' # Nama yang digunakan bebas
pickle.dump(logreg, open(filename, 'wb')) #write binary Membuat file model, ganti logreg dengan nama model yang digunakan
filename = 'scaler.pkl' # Nama yang digunakan bebas
pickle.dump(scaler, open(filename, 'wb')) #write binary Membuat file scaler, ganti scaler dengan nama scaler yang digunakan

In [30]:
feature_test

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
6,-1.506521,0.800654,-1.341272,-1.181504
111,0.674501,-0.819166,0.876490,0.922064
80,-0.416010,-1.513375,0.023504,-0.129720
52,1.280340,0.106445,0.649027,0.396172
86,1.038005,0.106445,0.535296,0.396172
23,-0.900681,0.569251,-1.170675,-0.918558
65,1.038005,0.106445,0.364699,0.264699
147,0.795669,-0.124958,0.819624,1.053537
9,-1.143017,0.106445,-1.284407,-1.444450
41,-1.627688,-1.744778,-1.398138,-1.181504


In [31]:
label_pred

array(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor',
       'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica',
       'Iris-setosa', 'Iris-versicolor'], dtype=object)

In [ ]:
# list1=label_pred.tolist()

In [ ]:
# test=json.dumps(list1)

In [ ]:
# print(list1)
# type(test)

# **API**

In [43]:
app = Flask(__name__)
@app.route("/home")
def home():
    html="""<!DOCTYPE html>
    <html>
    <body>

    <h1>Aplikasi Test</h1>
    <p>Test</p>

    </body>
    </html>"""

    return html

In [44]:
@app.route('/classify', methods=['POST'])
def classification():
    filename = 'model.pkl'
    loaded_model = pickle.load(open(filename, 'rb'))
    filename = 'scaler.pkl'
    loaded_scaler = pickle.load(open(filename, 'rb'))

    sepal_width = float(request.json['sepal_width'])
    sepal_length = float(request.json['sepal_length'])
    petal_length = float(request.json['petal_length'])
    petal_width = float(request.json['petal_width'])

    test_data=[[sepal_length,sepal_width,petal_length,petal_width]]
    feature_scaled=loaded_scaler.transform(test_data)
  
    prediction_example = loaded_model.predict(feature_scaled).tolist()
    # list1=prediction_example.tolist()
    # test=json.dumps(list1)
    
    return jsonify({
      "output": prediction_example
    })

# input data
# {
# 	"sepal_width": 3.5,
# 	"sepal_length": 5.1,
# 	"petal_width": 0.2,
# 	"petal_length": 1.4	
# }
# output:
# Iris-setosa

In [45]:
run_with_ngrok(app)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://778f00ee60fc.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Oct/2020 22:42:56] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2020 22:43:04] "POST /classify HTTP/1.1" 200 -



mengambil data pertama dari iris.csv
* SepalLengthCm:	 5.1
* SepalWidthCm:	 3.5
* PetalLengthCm:	 1.4
* PetalWidthCm:   0.2

hasil berdasarkan prediksi adalah Iris-setosa
Hasil dari sistem API juga sesuai